# 📘 Ejemplo de uso e interpretación del modelo LSTM + Word2Vec

# **1. Definición de librerias e instalaciones en general**

En esta sección se asegura la instalación de gensim, una librería esencial para trabajar con modelos Word2Vec. Aunque puede generar advertencias por conflictos de dependencias, la instalación finaliza correctamente para los fines del proyecto. A continuación, se importan las librerías necesarias para el uso e interpretación del modelo preentrenado: numpy para operaciones matriciales, tensorflow para cargar el modelo LSTM, gensim para trabajar con los vectores Word2Vec, y load_model desde Keras para cargar los modelos .h5

In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
# Cargar librerías
import numpy as np
import tensorflow as tf
from gensim.models import KeyedVectors
from tensorflow.keras.models import load_model

# **2. Montaje, carga de modelos en memoria**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Subir ambos modelos desde tu PC
from google.colab import files
uploaded = files.upload()  # Ejecutá esto y subí los dos archivos .h5

# Cargar ambos modelos
from tensorflow.keras.models import load_model

modelo_binario = load_model('lstm_word2vec_model.h5')
modelo_multiclase = load_model('lstm_word2vec_model_multiclase.h5')

print("✅ Modelos cargados correctamente.")


Saving lstm_word2vec_model_multiclase.h5 to lstm_word2vec_model_multiclase.h5
Saving lstm_word2vec_model.h5 to lstm_word2vec_model.h5


✅ Modelos cargados correctamente.


In [4]:
from gensim.models import KeyedVectors

w2v_model = KeyedVectors.load_word2vec_format(
    '/content/drive/MyDrive/Topicos Especiales 2/Proyecto Final/GoogleNews-vectors-negative300.bin',
    binary=True
)

print("✅ Modelo Word2Vec cargado exitosamente.")

✅ Modelo Word2Vec cargado exitosamente.


# **3. Función de predicción**

In [9]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

def vectorize_text(tokens, w2v_model, vector_size=300):
    vectors = []
    for word in tokens:
        if word in w2v_model:
            vectors.append(w2v_model[word])
    return vectors

def prepare_sequences(list_of_vectors, max_len=None):
    vector_sequences = [np.array(seq) for seq in list_of_vectors]
    if not vector_sequences:
        return None
    if max_len is None:
        max_len = max(len(seq) for seq in vector_sequences)

    padded = []
    for seq in vector_sequences:
        if len(seq) < max_len:
            pad_width = ((0, max_len - len(seq)), (0, 0))
            padded_seq = np.pad(seq, pad_width, mode='constant')
        else:
            padded_seq = seq[:max_len]
        padded.append(padded_seq)
    return np.array(padded)


# **4. Pruebas automáticas**

In [11]:
def predecir_texto(texto, modelo, w2v_model, max_len):
    """
    Preprocesa el texto, lo vectoriza, aplica padding y predice con el modelo entrenado.
    """
    tokens = preprocess_text(texto)
    vectorized = vectorize_text(tokens, w2v_model)
    if len(vectorized) == 0:
        return "❌ Ninguna palabra reconocida en Word2Vec."

    padded = prepare_sequences([vectorized], max_len=max_len)

    pred = modelo.predict(padded)
    clase_predicha = np.argmax(pred)
    probabilidad = pred[0][clase_predicha]

    etiquetas = {
        0: "Nobleza",
        1: "Transporte",
        2: "Ciudad",
        3: "Clima"
    }

    return f"🧠 Predicción: Clase {clase_predicha} ({etiquetas[clase_predicha]}) con probabilidad {probabilidad:.2%} de confianza"


In [27]:
ejemplos = [
    "The king is riding a horse.",
    "A bus is parked next to the garage.",
    "The mayor announced a new public transport system.",
    "Rain and storm covered the city during the night."
]

print("🔁 Pruebas automáticas con ambos modelos:\n")
for texto in ejemplos:
    print(f"📝 Texto: {texto}")
    print("🔷 Binario:", predecir_texto(texto, modelo_binario, w2v_model, max_len=15))
    print("🟣 Multiclase:", predecir_texto(texto, modelo_multiclase, w2v_model, max_len=15))
    print("-" * 60)


🔁 Pruebas automáticas con ambos modelos:

📝 Texto: The king is riding a horse.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
🔷 Binario: 🧠 Predicción: Clase 0 (Nobleza) con probabilidad 100.00% de confianza
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🟣 Multiclase: 🧠 Predicción: Clase 0 (Nobleza) con probabilidad 96.92% de confianza
------------------------------------------------------------
📝 Texto: A bus is parked next to the garage.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🔷 Binario: 🧠 Predicción: Clase 0 (Nobleza) con probabilidad 0.00% de confianza
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🟣 Multiclase: 🧠 Predicción: Clase 1 (Transporte) con probabilidad 99.99% de confianza
------------------------------------------------------------
📝 Texto: The mayor announced a new public transport system.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
🔷 Binario: 🧠 Predicción: Clase 0 (Nobleza) con probabilidad 33.14% de confianza
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🟣 Multiclase: 🧠 Predicción: Clase 2 (Ciudad) con probabil

# **5. Prueba interactiva**

In [ ]:
while True:
    texto = input("✍️ Escribí un texto para clasificar (o escribí 'salir' para terminar):\n")

    if texto.lower() == 'salir':
        print("✅ Clasificación finalizada.")
        break

    print("\n🔷 Predicción con modelo binario:")
    print(predecir_texto(texto, modelo_binario, w2v_model, max_len=15))

    print("\n🟣 Predicción con modelo multiclase:")
    print(predecir_texto(texto, modelo_multiclase, w2v_model, max_len=15))

    print("\n" + "-"*60)


✍️ Escribí un texto para clasificar (o escribí 'salir' para terminar):
My palace is covered by the heavy snow and rain

🔷 Predicción con modelo binario:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🧠 Predicción: Clase 0 (Nobleza) con probabilidad 100.00% de confianza

🟣 Predicción con modelo multiclase:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
🧠 Predicción: Clase 3 (Clima) con probabilidad 100.00% de confianza

------------------------------------------------------------
✍️ Escribí un texto para clasificar (o escribí 'salir' para terminar):
The royal family is visiting the city, thats why the city is covered with all kind of decorations

🔷 Predicción con modelo binario:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
🧠 Predicción: Clase 0 (Nobleza) con probabilidad 100.00% de confianza

🟣 Predicción con modelo multiclase:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🧠 Predicción: Clase 2 (Ciudad) con probabilidad 99.96% de confianza

------------------------------------------------------------
✍️ Escribí un text